In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install biobert-embedding==0.1.2

     |████████████████████████████████| 748.9MB 21kB/s 
     |████████████████████████████████| 133kB 57.8MB/s 
  Created wheel for biobert-embedding: filename=biobert_embedding-0.1.2-cp36-none-any.whl size=5702 sha256=de00aaa11b0146612ca59a8ff48674d579b7db109c61ebfb80351097fd1fced7
  Stored in directory: /root/.cache/pip/wheels/ad/15/65/3fc6192a7cb7920672bb46d566173decb0875f35bbe03cd09d
Successfully built biobert-embedding
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.2.0 which is incompatible.
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101


In [3]:
!pip install transformers

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 53.2MB/s 
     |████████████████████████████████| 3.8MB 61.0MB/s 
     |████████████████████████████████| 890kB 63.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7e934ec5e844ceebfafe87c6e502a8865a1b7fd8668b5ac1e6fe65569788e795
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/task2_en_training.tsv',delimiter='\t',encoding='utf-8')
df.loc[0]

tweet_id                     344266386467606528
user_id                               809439366
class                                         0
tweet       depression hurts, cymbalta can help
Name: 0, dtype: object

In [5]:
tweets = df.tweet.values
label = df[['class']]
labels =label.values
labels


array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("/content/gdrive/My Drive/Biobert/vocab.txt")

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [8]:
print(tweets[0])
print(tokenizer.tokenize(tweets[0]))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweets[0])))

depression hurts, cymbalta can help
['depression', 'hurts', ',', 'c', '##ym', '##balt', '##a', 'can', 'help']
[7560, 15483, 117, 172, 17162, 26443, 1161, 1169, 1494]


In [9]:
maxlen = 0
for t in tweets:
  ids = tokenizer.encode(t,add_special_tokens=True)
  maxlen = max(len(ids), maxlen)

print(maxlen)

82


In [0]:
input_ids = []
attention_masks = []
for t in tweets:
  encoded_dict = tokenizer.encode_plus(
                      t,
                      add_special_tokens = True,
                      max_length = maxlen,
                      pad_to_max_length = True,
                      return_attention_mask = True,
                      return_tensors = 'pt',
  )
  input_ids.append(encoded_dict['input_ids'])
    
  #attention masks
  attention_masks.append(encoded_dict['attention_mask'])
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [11]:
print(input_ids[0])

tensor([  101,  7560, 15483,   117,   172, 17162, 26443,  1161,  1169,  1494,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0])


In [0]:
from torch.utils.data import TensorDataset, random_split
dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [14]:
from transformers import BertModel, BertTokenizer, BertConfig, BertForSequenceClassification
config = BertConfig.from_json_file('/content/gdrive/My Drive/Biobert/bert_config.json')
model= BertForSequenceClassification.from_pretrained('/content/gdrive/My Drive/Biobert/model.ckpt-1000000.index'
                                                      ,from_tf=True, config=config)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

In [0]:
from transformers import get_linear_schedule_with_warmup


epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
device = torch.device("cuda")
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch_i in range(0, epochs):
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      model.zero_grad()
      loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
      total_train_loss += loss.item()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      scheduler.step()

In [0]:
df_stats = pd.DataFrame(data=training_stats)
df_stats

In [20]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv('/content/gdrive/My Drive/task2_en_validation.tsv',delimiter='\t',encoding='utf-8')

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))


tweets = df.tweet.values
label = df[['class']]
labels =label.values
labels

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in tweets:
  
    encoded_dict = tokenizer.encode_plus(
                        sent,                     
                        add_special_tokens = True,
                        max_length = 82,          
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',     
                   )
       
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

batch_size = 32  

prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


Number of test sentences: 5,134



In [21]:

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  
  batch = tuple(t.to(device) for t in batch)
  
  
  b_input_ids, b_input_mask, b_labels = batch
  
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 5,134 test sentences...
    DONE.


In [0]:
all_pred = []
for batch in (predictions):
  for pred in batch:
    all_pred.append(pred)


In [0]:
all_pred_label = []
for i in all_pred:
  all_pred_label.append(np.argmax(i))

In [24]:
from sklearn.metrics import classification_report
print(classification_report(df['class'],all_pred_label))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95      4660
           1       0.54      0.65      0.59       474

    accuracy                           0.92      5134
   macro avg       0.75      0.80      0.77      5134
weighted avg       0.92      0.92      0.92      5134

